In [1]:
# Also see https://github.com/shubhamagarwal92/visdial_conv/blob/master/notebooks/acl_analysis.ipynb
import sys
sys.path.append("../")

In [2]:
from autoreview.models.langchain_openai_agent import OpenAIAgent

/mnt/home/miniconda/envs/autoreview/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
# from autoreview.models.langchain_openai_agent import OpenAIAgent
from autoreview.models.data_utils import (load_all_prompts) 
from autoreview.models.anyscale_endpoint import anyscale_chat_complete

In [4]:
prompts = load_all_prompts()
print(prompts.keys())

dict_keys(['research_template', 'llama_template', 'abstract_template', 'per_cite_template', 'cot_template', 'cot_v1_template', 'cot_v0_template', 'cot_v2_template', 'vanilla_template', 'vanilla_template_full_text', 'plan_template', 'plan_-1_template', 'zero_chain_thought_template', 'plan_learned_template', 'suffix_plan_learned_template', 'per_sentence_template'])


In [5]:
print(prompts["vanilla_template"])
print("\n")
print(prompts["plan_template"])

You will be provided with an abstract of a scientific document and other references papers in triple quotes. Your task is to write the related work section of the document using only the provided abstracts and other references papers. Please write the related work section creating a cohesive storyline by doing a critical analysis of prior work comparing the strengths and weaknesses while also motivating the proposed approach. You should cite the other related documents as (@cite_#) whenever it is referred in the related work. Do not cite abstract. Do not include any extra notes or newline characters at the end. Do not copy the abstracts of reference papers directly but compare and contrast to the main work concisely. Do not provide the output in bullet points. Do not provide references at the end. Provide the output in max 200 words.


You will be provided with an abstract of a scientific document and other references papers in triple quotes. Your task is to write the related work sect

In [6]:
main_abstract = """We explore the zero-shot abilities of recent large language models (LLMs) for the task of writing the literature review of a scientific research paper conditioned on
its abstract and the content of related papers. We propose and examine a novel strategy for literature review generation with an LLM in which we first generate a
plan for the review, and then use it to generate the actual text. While modern LLMs can easily be trained or prompted to condition on all abstracts of papers to be cited
to generate a literature review without such intermediate plans, our empirical study shows that these intermediate plans improve the quality of generated literature
reviews over vanilla zero-shot generation. Furthermore, we also create a new test corpus consisting of recent arXiv papers (with full content)
posted after both open-sourced and closed-sourced LLMs that were used in our study were released. This allows us to ensure that our zero-shot experiments do 
not suffer from test set contamination"""

In [7]:
cite1_abstract = """ Multi-document summarization is a challenging task for which there exists little largescale datasets. We propose Multi-XScience, a large-scale multi-document summarization
dataset created from scientific articles. MultiXScience introduces a challenging multidocument summarization task: writing the related-work section of a paper based on its abstract and the articles it references. Our
work is inspired by extreme summarization, a dataset construction protocol that favours abstractive modeling approaches. Descriptive statistics and empirical results—using several state-of-the-art models trained on the MultiXScience dataset—reveal that Multi-XScience
is well suited for abstractive models"""

In [8]:
cite2_abstract = """We present a method to produce abstractive summaries of long documents that exceed several thousand words via neural abstractive summarization. We perform a simple extractive step before generating a summary, which is then used to condition the transformer language model on relevant information before being tasked with generating a summary. We show that this extractive step significantly improves summarization results. We also show that this approach produces more abstractive summaries compared to prior work that employs a copy mechanism while still achieving higher rouge scores. Note: The abstract above was not written by the authors, 
it was generated by one of the models presented in this paper."""

 # https://arxiv.org/abs/2304.08637

In [9]:
cite3_abstract = """ Information overload is a major obstacle to scientific progress. The explosive growth in scientific literature and data has made it ever harder
 to discover useful insights in a large mass of information. Today scientific knowledge is accessed through search engines, but they are unable to organize 
 scientific knowledge alone. In this paper we introduce Galactica: a large language model that can store, combine and reason about scientific knowledge. 
 We train on a large scientific corpus of papers, reference material, knowledge bases and many other sources. We outperform existing models on a 
 range of scientific tasks. On technical knowledge probes such as LaTeX equations, Galactica outperforms the latest GPT-3 by 68.2% versus 49.0%. 
 Galactica also performs well on reasoning, outperforming Chinchilla on mathematical MMLU by 41.3% to 35.7%, and PaLM 540B on MATH with a score of 
 20.4% versus 8.8%. It also sets a new state-of-the-art on downstream tasks such as PubMedQA and MedMCQA dev of 77.6 and 52.9%. And despite not b
 eing trained on a general corpus, Galactica outperforms BLOOM and OPT-175B on BIG-bench. We believe these results demonstrate the potential for 
 language models as a new interface for science. We open source the model for the benefit of the scientific community.
"""

In [12]:
vanilla_template = """You will be provided with an abstract of a scientific document and other references papers in triple quotes. 
Your task is to write the related work section of the document using only the provided abstracts and other references papers. Please write the 
related work section creating a cohesive storyline by doing a critical analysis of prior work comparing the strengths and weaknesses while also motivating the proposed approach. 
You should cite the other related documents as (@cite_#) whenever it is referred in the related work. Do not cite abstract. 
Do not include any extra notes or newline characters at the end. Do not copy the abstracts of reference papers directly but compare and contrast to the main work concisely. 
Do not provide the output in bullet points. Do not provide references at the end. Provide the output in max 80 words."""


In [13]:
plan = "Please generate 5 sentences in 120 words. Cite @cite_1 at line 1 and 3. Cite @cite_2 at line 2 and 5. Cite @cite_3 at line 4 and 5."

In [14]:

# base_prompt = prompts["vanilla_template"]
base_prompt = prompts["vanilla_template"]
base_prompt = base_prompt.replace("200", "60")
complete_prompt = "{base_prompt}\n\n```Main abstract: {abstract} \n\n Reference @cite_1: {cite1_abstract}\n\nReference @cite_2: {cite2_abstract}\n\nReference  @cite_3: {cite3_abstract}```` Related Work:".format(
    base_prompt=base_prompt, abstract=main_abstract,  cite1_abstract=cite1_abstract, cite2_abstract=cite2_abstract, cite3_abstract=cite3_abstract)
base_prompt = prompts["plan_template"]
complete_plan_prompt = "{base_prompt}\n\n```Main abstract: {abstract} \n\n Reference @cite_1: {cite1_abstract}\n\nReference @cite_2: {cite2_abstract}\n\nReference  @cite_3: {cite3_abstract} \n\n Plan: {plan}```` Related Work:\n".format(
    base_prompt=base_prompt, abstract=main_abstract, cite="@cite_1", cite1_abstract=cite1_abstract, cite2_abstract=cite2_abstract, cite3_abstract=cite3_abstract, plan=plan)
# complete_plan_prompt = "{base_prompt}\n\n```Main abstract: {abstract} \n\n Reference {cite}: {ref_abstract} \n\n Plan: {plan}```` Related Work:\n".format(
#     base_prompt=base_prompt, abstract=main_abstract, cite="@cite_1", ref_abstract=ref_abstract, plan=plan)

In [15]:
print(complete_prompt)

You will be provided with an abstract of a scientific document and other references papers in triple quotes. Your task is to write the related work section of the document using only the provided abstracts and other references papers. Please write the related work section creating a cohesive storyline by doing a critical analysis of prior work comparing the strengths and weaknesses while also motivating the proposed approach. You should cite the other related documents as (@cite_#) whenever it is referred in the related work. Do not cite abstract. Do not include any extra notes or newline characters at the end. Do not copy the abstracts of reference papers directly but compare and contrast to the main work concisely. Do not provide the output in bullet points. Do not provide references at the end. Provide the output in max 60 words.

```Main abstract: We explore the zero-shot abilities of recent large language models (LLMs) for the task of writing the literature review of a scientific 

In [16]:
print(complete_plan_prompt)

You will be provided with an abstract of a scientific document and other references papers in triple quotes. Your task is to write the related work section of the document using only the provided abstracts and other references papers. Please write the related work section creating a cohesive storyline by doing a critical analysis of prior work comparing the strengths and weaknesses while also motivating the proposed approach. You are also provided a plan mentioning the total number of lines and the citations to refer in different lines. You should cite the other related documents as (@cite_#) whenever it is referred in the related work. Do not cite abstract. Do not include any extra notes or newline characters at the end. Do not copy the abstracts of reference papers directly but compare and contrast to the main work concisely. Do not provide the output in bullet points. Do not provide references at the end. Please follow the plan when generating sentences, especially the number of lin

GPT-4

In [17]:
# model_name = "gpt-3.5-turbo"
model_name = "gpt-4"
ml_model = OpenAIAgent(model_name)

Running the experiment for gpt-4 using Langchain


In [18]:

response = ml_model.get_response(complete_prompt)["response"]
print(response)

The literature review generation task, a form of multi-document summarization, has been tackled through different strategies. @cite_1 proposed a dataset, MultiXScience, to foster abstractive approaches, while @cite_2 used an extractive step to enhance neural summarization. Our work complements these by introducing intermediate plans to improve generation quality. Meanwhile, @cite_3's Galactica demonstrated potential for language models in scientific tasks, a direction we explore with zero-shot experiments.


In [19]:
response = ml_model.get_response(complete_plan_prompt)["response"]
print(response)

The concept of generating the related-work section from abstracts and referenced articles is not entirely new, as evidenced by the Multi-XScience dataset proposed by @cite_1. The approach of extending abstractive summarization to lengthy documents via an extractive step for conditioning, as described by @cite_2, aligns with the proposed methodology here. However, the strategy introduced in this study differentiates itself by creating an iterative plan prior to the generation of the review, a step not present in the work of @cite_1. The LLM used in this study also builds upon the capabilities of models like Galactica @cite_3, which have demonstrated exceptional performance on scientific tasks. The results of @cite_2 and @cite_3 support the potential of large language models in scientific literature review generation, while the proposed approach aims to further enhance the quality of these outputs through structured planning.


GPT-3.5

In [20]:
import os
import openai
API_KEY = os.environ["OPENAI_API_KEY"]
openai.api_key = API_KEY

In [21]:
model_name = "gpt-3.5-turbo"
# model_name = "gpt-4"
ml_model = OpenAIAgent(model_name)

Running the experiment for gpt-3.5-turbo using Langchain


In [22]:
response = ml_model.get_response(complete_prompt)["response"]
print(response)
print(f"Total length of response: {len(response)}")

Prior work has explored the use of large language models (LLMs) for various natural language processing tasks. Multi-document summarization is one such task, and the Multi-XScience dataset (@cite_1) has been created specifically for this purpose. This dataset focuses on writing the related-work section of a scientific paper based on its abstract and the articles it references. The dataset construction protocol favors abstractive modeling approaches, making it suitable for LLMs.

Another approach to abstractive summarization is presented in a paper (@cite_2), where the authors propose a method to generate summaries of long documents. They employ an extractive step to gather relevant information before conditioning the transformer language model for summary generation. This approach yields more abstractive summaries compared to prior work, while still achieving higher rouge scores.

Furthermore, the concept of using LLMs for scientific knowledge organization and reasoning is introduced i

In [23]:
response = ml_model.get_response(complete_plan_prompt)["response"]
print(response)

The task of generating a literature review using large language models (LLMs) has gained attention recently. One approach is to condition the LLM on the abstract and content of related papers to generate the review (@cite_1). However, it has been observed that generating an intermediate plan for the review improves its quality compared to vanilla zero-shot generation. This motivated the proposal and examination of a novel strategy where a plan is first generated and then used to generate the actual text (@cite_1). Another method to improve abstractive summarization is by performing an extractive step before generating the summary. This step conditions the transformer language model on relevant information, leading to better summarization results compared to prior work that employs a copy mechanism (@cite_2). Additionally, the use of large language models like Galactica has shown promising results in organizing and reasoning about scientific knowledge (@cite_3). Galactica outperforms ex

The following would replace the OpenAI key with Anyscale key

In [24]:
engine = "meta-llama/Llama-2-70b-chat-hf"
response = anyscale_chat_complete(prompt=complete_prompt, engine=engine)
print(response)

In this section, we discuss the related work in the field of scientific literature review generation using large language models (LLMs). Recent works have explored the use of LLMs for multi-document summarization tasks, such as generating the related work section of a paper based on its abstract and the articles it references (@cite_1). These works have shown that LLMs can produce high-quality summaries, but may suffer from the lack of relevant information and context (@cite_2).

To address this issue, we propose a novel strategy for literature review generation that involves generating a plan for the review before generating the actual text. This approach has been shown to improve the quality of generated literature reviews over vanilla zero-shot generation (@cite_3). Additionally, we create a new test corpus consisting of recent arXiv papers to ensure that our zero-shot experiments do not suffer from test set contamination.

Our work builds upon the recent advances in LLMs and their 

In [26]:
response = anyscale_chat_complete(prompt=complete_plan_prompt, engine=engine)
print(response)

In recent years, there has been a growing interest in using large language models (LLMs) for scientific literature review generation (@cite_1). One approach is to use a novel strategy that involves generating a plan for the review and then using it to generate the actual text (@cite_1). This approach has been shown to improve the quality of generated literature reviews over vanilla zero-shot generation (@cite_1). Another approach is to use a simple extractive step before generating a summary, which is then used to condition the transformer language model on relevant information (@cite_2). This approach has been shown to produce more abstractive summaries compared to prior work that employs a copy mechanism while still achieving higher ROUGE scores (@cite_2). Additionally, there have been efforts to train LLMs on large scientific corpora to improve their ability to store, combine, and reason about scientific knowledge (@cite_3). For example, Galactica, a large language model, has been t

In [ ]:
# response.replace("@cite_1", "\\citep{lu-etal-2020-multi-xscience}")
# % \textcolor{blue}{text}\\
# % \textcolor{red}{first color}\\
# % \textcolor{orange}{data} \\
# % \textcolor{red}{sample}  \textcolor{purple}{tet}\\
# % \textcolor{purple}{another} \textcolor{purple}{one} 
